In [87]:
input_test = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

with open('data/day_5.txt') as txtfile:
    input_day = txtfile.read()

In [363]:
import re
import pandas as pd
PROD = 1
input_data = input_day if PROD else input_test
list_maps = ["seed-to-soil map", "soil-to-fertilizer map", "fertilizer-to-water map", "water-to-light map",
             "light-to-temperature map", "temperature-to-humidity map", "humidity-to-location map", ""]
dicc_maps = {"seeds": [int(i) for i in re.findall(r'(?<=seeds:\s)(?=(.*\d+))', input_data)[0].split()]}
for pos, l in enumerate(list_maps[:-1]):
    dicc_maps[l] = [[int(j) for j in i.split()] for i in re.findall(f'(?<={l}:\s)([\d\s]+)(?={list_maps[pos + 1]})', input_data)[0].splitlines() if i]

dicc_maps

{'seeds': [1044452533,
  40389941,
  3710737290,
  407166728,
  1552449232,
  639689359,
  3327654041,
  26912583,
  3440484265,
  219136668,
  1126550158,
  296212400,
  2332393052,
  229950158,
  200575068,
  532702401,
  4163696272,
  44707860,
  3067657312,
  45353528],
 'seed-to-soil map': [[1953514507, 1808056938, 198190267],
  [3721110896, 2006247205, 109062451],
  [1046714200, 3920284337, 339096560],
  [965003502, 2389762347, 16609477],
  [1946017538, 563731936, 3971811],
  [2151704774, 1532707249, 120594830],
  [1385810760, 3525158, 560206778],
  [1949989349, 0, 3525158],
  [3830173347, 2115309656, 274452691],
  [0, 567703747, 902873308],
  [2272299604, 2471473045, 1448811292],
  [902873308, 1470577055, 62130194],
  [981612979, 2406371824, 65101221],
  [4104626038, 1653302079, 154754859]],
 'soil-to-fertilizer map': [[131659986, 0, 297180572],
  [86587273, 297180572, 45072713],
  [1582904051, 2761968868, 319965883],
  [2557550592, 597853135, 123775334],
  [3043086384, 12155104

# Part 1

In [267]:
def get_destination(value, origins):
    for k, v in origins.items():
        if value in k:
            return value + v
    return value

location_values = []
for value in dicc_maps["seeds"]:
    for m in list_maps[:-1]:
        origins = [o[1] for o in dicc_maps[m]]
        destinations = [o[0] for o in dicc_maps[m]]
        od_steps = [o[2] for o in dicc_maps[m]]

        o_intervals = {}
        for o, d, s in zip(origins, destinations, od_steps) :
            o_intervals[pd.Interval(o, o + s - 1, closed='both')] = d - o

        value = get_destination(value, o_intervals)
    location_values.append(value)
print(min(location_values))

836040384


# Part 2

In [299]:
m = "seed-to-soil map"
origins = [o[1] for o in dicc_maps[m]]
destinations = [o[0] for o in dicc_maps[m]]
od_steps = [o[2] for o in dicc_maps[m]]
o_intervals = {}
for o, d, s in zip(origins, destinations, od_steps) :
    o_intervals[pd.Interval(o, o + s - 1, closed='both')] = d - o
o_intervals = {k: o_intervals[k] for k in sorted(o_intervals)}
o_intervals

{Interval(0, 3525157, closed='both'): 1949989349,
 Interval(3525158, 563731935, closed='both'): 1382285602,
 Interval(563731936, 567703746, closed='both'): 1382285602,
 Interval(567703747, 1470577054, closed='both'): -567703747,
 Interval(1470577055, 1532707248, closed='both'): -567703747,
 Interval(1532707249, 1653302078, closed='both'): 618997525,
 Interval(1653302079, 1808056937, closed='both'): 2451323959,
 Interval(1808056938, 2006247204, closed='both'): 145457569,
 Interval(2006247205, 2115309655, closed='both'): 1714863691,
 Interval(2115309656, 2389762346, closed='both'): 1714863691,
 Interval(2389762347, 2406371823, closed='both'): -1424758845,
 Interval(2406371824, 2471473044, closed='both'): -1424758845,
 Interval(2471473045, 3920284336, closed='both'): -199173441,
 Interval(3920284337, 4259380896, closed='both'): -2873570137}

In [364]:
def intersect_intervals(interval_1, interval_2):
    intervals = []
    left_interval = pd.Interval(min(interval_1.left, interval_2.left), max(interval_1.left, interval_2.left), closed="left")
    mid_interval = pd.Interval(max(interval_1.left, interval_2.left), min(interval_1.right, interval_2.right), closed="both")
    right_interval = pd.Interval(min(interval_1.right, interval_2.right), max(interval_1.right, interval_2.right), closed="right")
    intervals = [i for i in set([left_interval, mid_interval, right_interval]) if interval_1.overlaps(i)]
    return intervals


def get_intervals_destinations(input_interval, origins):
    value_intervals = []
    for k, v in origins.items():
        if input_interval.overlaps(k):
            value_interval = input_interval + v
            intersected_intervals = intersect_intervals(input_interval, k)
            for j in intersected_intervals:
                if j.closed!="both":
                    seed_interval = j
                else:
                    value_intervals.append(value_interval)
    if value_intervals:
        return value_intervals
    else:
        return [input_interval]

In [382]:
# for i in range(0, len(dicc_maps["seeds"]), 2)[0]:
#     seed_init = dicc_maps["seeds"][i]
#     seed_end = dicc_maps["seeds"][i + 1]
i = 4
seed_init = dicc_maps["seeds"][i]
seed_end = dicc_maps["seeds"][i + 1]
location_intervals = []
input_interval = [pd.Interval(seed_init, seed_init + seed_end-1, closed="both")]
for m in list_maps[1:-1]:
    origins = [o[1] for o in dicc_maps[m]]
    destinations = [o[0] for o in dicc_maps[m]]
    od_steps = [o[2] for o in dicc_maps[m]]
    o_intervals = {}
    for o, d, s in zip(origins, destinations, od_steps) :
        o_intervals[pd.Interval(o, o + s - 1, closed='both')] = d - o
    o_intervals = {k: o_intervals[k] for k in sorted(o_intervals)}
    #print(o_intervals)
    new_intervals = []
    for j in input_interval:
        new_intervals += get_intervals_destinations(j, o_intervals)
    input_interval = new_intervals
location_intervals += input_interval
location_intervals   

[Interval(3827907647, 4467597005, closed='both'),
 Interval(1467047206, 2106736564, closed='both'),
 Interval(4063682656, 4703372014, closed='both'),
 Interval(1702822215, 2342511573, closed='both'),
 Interval(2374809631, 3014498989, closed='both'),
 Interval(2736617106, 3376306464, closed='both'),
 Interval(1892103837, 2531793195, closed='both'),
 Interval(2283913395, 2923602753, closed='both'),
 Interval(2955900811, 3595590169, closed='both'),
 Interval(3317708286, 3957397644, closed='both'),
 Interval(2473195017, 3112884375, closed='both'),
 Interval(2473195017, 3112884375, closed='both'),
 Interval(3996835553, 4636524911, closed='both'),
 Interval(1635975112, 2275664470, closed='both'),
 Interval(305486450, 945175808, closed='both'),
 Interval(2392002300, 3031691658, closed='both'),
 Interval(2392002300, 3031691658, closed='both'),
 Interval(3289026493, 3928715851, closed='both'),
 Interval(2241750644, 2881440002, closed='both'),
 Interval(2241750644, 2881440002, closed='both'),
 I

[Interval(2353608708, 2393998648, closed='both')]

In [367]:
min([i for i in location_intervals if i.left>0]).left + 1

405301

45

In [ ]:
#Too high
2784810021

#too low
405301

In [314]:
get_intervals_destinations(j, o_intervals)

[Interval(2868483871, 2913837398, closed='both')]

In [324]:
m = "seed-to-soil map"
origins = [o[1] for o in dicc_maps[m]]
destinations = [o[0] for o in dicc_maps[m]]
od_steps = [o[2] for o in dicc_maps[m]]
o_intervals = {}
for o, d, s in zip(origins, destinations, od_steps) :
    o_intervals[pd.Interval(o, o + s - 1, closed='both')] = d - o
o_intervals = {k: o_intervals[k] for k in sorted(o_intervals)}

soils = []
for i in range(0, len(dicc_maps["seeds"]), 2):
    seed_init = dicc_maps["seeds"][i]
    seed_end = dicc_maps["seeds"][i + 1]
    input_interval = [pd.Interval(seed_init, seed_init + seed_end-1, closed="both")]
    for j in input_interval:
        print(get_intervals_destinations(j, o_intervals))

[Interval(476748786, 517138726, closed='both')]
[Interval(3511563849, 3918730576, closed='both'), Interval(837167153, 1244333880, closed='both')]
[Interval(2171446757, 2811136115, closed='both'), Interval(4003773191, 4643462549, closed='both'), Interval(1697906801, 2337596159, closed='both'), Interval(3267312923, 3907002281, closed='both'), Interval(3267312923, 3907002281, closed='both')]
[Interval(3128480600, 3155393182, closed='both')]
[Interval(3241310824, 3460447491, closed='both')]
[Interval(558846411, 855058810, closed='both')]
[Interval(4047256743, 4277206900, closed='both'), Interval(907634207, 1137584364, closed='both'), Interval(907634207, 1137584364, closed='both'), Interval(2133219611, 2363169768, closed='both')]
[Interval(1582860670, 2115563070, closed='both'), Interval(1582860670, 2115563070, closed='both'), Interval(-367128679, 165573721, closed='both')]
[Interval(1290126135, 1334833994, closed='both')]
[Interval(2868483871, 2913837398, closed='both')]


In [204]:
import numpy as np
import pandas as pd

In [212]:
pd.interval_range(1, 10, 1)

IntervalIndex([(1, 10]],
              closed='right',
              dtype='interval[int64]')

In [220]:
df = pd.DataFrame(data=np.random.rand(10, 2), columns=["a", "b"], index=pd.interval_range(1, 100, 10))
df2 = pd.DataFrame(data=np.random.rand(10, 2), columns=["c", "d"], index=pd.interval_range(1, 10, 10))
df

,a,b
"(1.0, 10.9]",0.130224,0.303502
"(10.9, 20.8]",0.145151,0.259462
"(20.8, 30.700000000000003]",0.242704,0.832309
"(30.700000000000003, 40.6]",0.350704,0.441557
"(40.6, 50.5]",0.338257,0.302209
"(50.5, 60.400000000000006]",0.664465,0.140064
"(60.400000000000006, 70.3]",0.490125,0.637123
"(70.3, 80.2]",0.473823,0.090151
"(80.2, 90.10000000000001]",0.580836,0.791029
"(90.10000000000001, 100.0]",0.428774,0.946875


In [222]:
df2

,c,d
"(1.0, 1.9]",0.625072,0.595591
"(1.9, 2.8]",0.518753,0.169920
"(2.8, 3.7]",0.798237,0.691173
"(3.7, 4.6]",0.447262,0.841375
"(4.6, 5.5]",0.020563,0.621798
"(5.5, 6.4]",0.086600,0.942272
"(6.4, 7.3]",0.287288,0.309455
"(7.3, 8.2]",0.083877,0.340327
"(8.2, 9.1]",0.070799,0.526806
"(9.1, 10.0]",0.132657,0.442366


In [221]:
df.join(df2)

,a,b,c,d
"(1.0, 10.9]",0.130224,0.303502,NaN,NaN
"(10.9, 20.8]",0.145151,0.259462,NaN,NaN
"(20.8, 30.700000000000003]",0.242704,0.832309,NaN,NaN
"(30.700000000000003, 40.6]",0.350704,0.441557,NaN,NaN
"(40.6, 50.5]",0.338257,0.302209,NaN,NaN
"(50.5, 60.400000000000006]",0.664465,0.140064,NaN,NaN
"(60.400000000000006, 70.3]",0.490125,0.637123,NaN,NaN
"(70.3, 80.2]",0.473823,0.090151,NaN,NaN
"(80.2, 90.10000000000001]",0.580836,0.791029,NaN,NaN
"(90.10000000000001, 100.0]",0.428774,0.946875,NaN,NaN


In [228]:
pd.Interval(70.5, 70.7)

Interval(70.5, 70.7, closed='right')

In [236]:
df.index.overlaps(pd.Interval(70.5, 70.7))

array([False, False, False, False, False, False, False,  True, False,
       False])